# Politics in Wesleyan

In [1]:
SCHOOL="wesleyan"
SUBJECT="opinion"
START_YEAR=2003
FINAL_YEAR=2023
DATA_DIR="data"      # should be 'data'
OUTPUT_DIR="output"  # should be 'output'

In [2]:
%pwd

'C:\\Users\\ranar\\ASDRP\\diversity-colleges\\notebooks'

In [3]:
%cd ..

C:\Users\ranar\ASDRP\diversity-colleges


In [4]:
import sys
sys.path.append('src')
import ouraws
import ourgraphs
import textutil

In [5]:
# NOTE: before loading, we need to be in the "diversity-colleges" folder

S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [6]:
df.shape

(1632, 6)

In [7]:
df.head(2)

,title,url,body,year,month,day
0,Why No One Needs to Come to Ariana DeBose’s Re...,http://wesleyanargus.com/2023/03/02/why-no-one...,Why No One Needs to Come to Ariana DeBose’s Re...,2023,3,2
1,How Wesleyan Makes Art Inaccessible,http://wesleyanargus.com/2023/03/02/how-wesley...,How Wesleyan Makes Art Inaccessible\nHow Wesle...,2023,3,2


In [8]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API_KEY")

BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

In [9]:
BIPARTISAN_API_KEY

'gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g=='

In [10]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

1.622514


In [11]:
df['year'].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013,
       2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003],
      dtype=int64)

In [12]:
# SYNCHRONOUS METHOD : TAKES A LONG TIME

results = []

for year in range(2010, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2010 ... 107
2010	107	157.571971	1.4726352429906542
Year: 2011 ... 124
2011	124	87.91005200000001	0.7089520322580646
Year: 2012 ... 75
2012	75	68.67182700000004	0.9156243600000005
Year: 2013 ... 86
2013	86	71.811564	0.8350181860465117
Year: 2014 ... 103
2014	103	74.349949	0.7218441650485437
Year: 2015 ... 109
2015	109	74.00612199999998	0.6789552477064218
Year: 2016 ... 108
2016	108	52.46216600000001	0.48576079629629637
Year: 2017 ... 96
2017	96	107.198201	1.1166479270833334
Year: 2018 ... 76
2018	76	121.471127	1.5983043026315789
Year: 2019 ... 76
2019	76	106.678957	1.4036704868421053
Year: 2020 ... 49
2020	49	17.905312	0.3654145306122449
Year: 2021 ... 50
2021	50	51.501996999999996	1.03003994
Year: 2022 ... 48
2022	48	49.741421000000024	1.036279604166667


In [13]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)